In [10]:
import tensorflow as tf
import numpy as np
import Image
import glob
import cv2

from random import randint, uniform
from scipy.misc.pilutil import imshow
from scipy.misc import imresize

In [2]:
def one_hot(label, size):
    arr = np.zeros(size)
    arr[:, label] = 1
    return arr


In [17]:
# Load training sets
red_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/train/0/*')
yellow_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/train/1/*')
green_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/train/2/*')
unknown_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/train/3/*')

red_features = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in red_filelist])
red_labels = one_hot(0, (red_features.shape[0], 4))

yellow_features = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in yellow_filelist])
yellow_labels = one_hot(1, (yellow_features.shape[0], 4))

green_features = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in green_filelist])
green_labels = one_hot(2, (green_features.shape[0], 4))

unknown_features = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in unknown_filelist])
unknown_labels = one_hot(3, (unknown_features.shape[0], 4))

features = np.concatenate([red_features, yellow_features, green_features, unknown_features], axis=0)
labels = np.concatenate([red_labels, yellow_labels, green_labels, unknown_labels], axis=0)

In [18]:
# Load test sets
# Real Test Images
red_tr_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/real/0/*')
red_features_tr = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in red_tr_filelist])
red_labels_tr = one_hot(0, (red_features_tr.shape[0], 4))

yellow_tr_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/real/1/*')
yellow_features_tr = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in yellow_tr_filelist])
yellow_labels_tr = one_hot(0, (yellow_features_tr.shape[0], 4))

green_tr_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/real/2/*')
green_features_tr = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in green_tr_filelist])
green_labels_tr = one_hot(0, (green_features_tr.shape[0], 4))

unknown_tr_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/real/3/*')
unknown_features_tr = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in unknown_tr_filelist])
unknown_labels_tr = one_hot(0, (unknown_features_tr.shape[0], 4))

# Simulated Test Images
red_ts_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/sim/0/*')
red_features_ts = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in red_ts_filelist])
red_labels_ts = one_hot(0, (red_features_ts.shape[0], 4))

yellow_ts_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/sim/1/*')
yellow_features_ts = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in yellow_ts_filelist])
yellow_labels_ts = one_hot(0, (yellow_features_ts.shape[0], 4))

green_ts_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/sim/2/*')
green_features_ts = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in green_ts_filelist])
green_labels_ts = one_hot(0, (green_features_ts.shape[0], 4))

unknown_ts_filelist = glob.glob('/home/paperspace/CarND-Capstone/ros/src/tl_detector/tl_classifier_data/test/sim/3/*')
unknown_features_ts = np.array([imresize(np.array(Image.open(fname)), (64, 96, 3)) for fname in unknown_ts_filelist])
unknown_labels_ts = one_hot(0, (unknown_features_ts.shape[0], 4))


In [23]:
# Data augmentation function
def probably_adjust_brightness(img):
    gamma = uniform(0.5, 2.0)
    inv_gamma = 1. / gamma
    lookup_table = np.array([((i / 255.) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, lookup_table)

def maybe_flip_horizontal(img):
    if randint(0, 1) == 0:
        return cv2.flip(img, 1)
    return img

def probably_translate(img):
    vertical_shift = randint(-10, 10)
    img = np.roll(img, vertical_shift, axis=0)
    
    horizontal_shift = randint(-10, 10)
    img = np.roll(img, horizontal_shift, axis=1)
    return img

def distort_img(img):
    new_img = np.uint8(img.copy())
    new_img = probably_translate(new_img)
    new_img = probably_adjust_brightness(new_img)
    new_img = maybe_flip_horizontal(new_img)
    return new_img

def get_batch(batch_size, seed_imgs, labels):
    num_seed_imgs = seed_imgs.shape[0]
    assert batch_size % num_seed_imgs == 0
    aug_factor = batch_size / num_seed_imgs
    
    batch_img = np.empty((batch_size, 64, 96, 3))
    batch_label = np.zeros((batch_size, 4))
    
    batch_index = 0
    for i in range(num_seed_imgs):
        curr_img = seed_imgs[i]
        curr_label = np.argmax(labels[i])
        for j in range(aug_factor):
            batch_img[batch_index, :] = distort_img(curr_img)
            batch_label[batch_index, curr_label] = 1
            batch_index += 1
            
    return batch_img, batch_label

In [55]:
# Model

class ConvNet:
    def __init__(self):
        self.x = tf.placeholder(tf.float32, shape=[None, 64, 96, 3], name='x')
        self.y = tf.placeholder(tf.float32, shape=[None, 4], name='y')
        self.phase = tf.placeholder(tf.bool, name='phase')
        
        # Layers of NN
        self.layer1 = self.conv_layer(self.x, 3, 8, self.phase, 'layer1')
        self.layer2 = self.conv_layer(self.layer1, 3, 8, self.phase, 'layer2')
        self.layer2_pool = tf.nn.max_pool(self.layer2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        self.layer3 = self.conv_layer(self.layer2_pool, 3, 16, self.phase, 'layer3')
        self.layer4 = self.conv_layer(self.layer3, 3, 16, self.phase, 'layer4')
        self.layer4_pool = tf.nn.max_pool(self.layer4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        self.layer5 = self.conv_layer(self.layer4_pool, 3, 32, self.phase, 'layer5')
        self.layer6 = self.conv_layer(self.layer5, 3, 32, self.phase, 'layer6')
        self.layer6_pool = tf.nn.max_pool(self.layer6, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        self.debug_shape = tf.shape(self.layer6_pool)
        flatten_size = 8 * 12 * 32
        self.flatten = tf.reshape(self.layer6_pool, [-1, flatten_size])
        self.layer7 = self.fc_layer(self.flatten, 1000, self.phase, 'layer7')
        self.layer8 = self.fc_layer(self.layer7, 500, self.phase, 'layer8')
        self.layer9 = self.fc_layer(self.layer8, 100, self.phase, 'layer9')
        self.logits = tf.contrib.layers.fully_connected(self.layer9, 4, activation_fn=None, scope='logits')
        
        self.loss = self.setup_loss(self.logits, self.y)
        self.accuracy = self.setup_accuracy(self.logits, self.y)
        self.inference = tf.argmax(self.logits, 1)
        self.train_op = tf.train.AdamOptimizer(1e-4).minimize(self.loss)
        
    def fc_layer(self, x, hh_size, phase, scope):
        with tf.variable_scope(scope):
            h1 = tf.contrib.layers.fully_connected(x, hh_size, activation_fn=None, scope='fully_connected')
            #h2 = tf.contrib.layers.batch_norm(h1, center=True, scale=True, is_training=phase, scope='bn')
            return tf.nn.relu(h1, 'relu')
        
    def conv_layer(self, x, kernel_size, filter_size, phase, scope):
        with tf.variable_scope(scope):
            h1 = tf.contrib.layers.conv2d(x, filter_size, kernel_size, activation_fn=None, scope='convolution')
            #h2 = tf.contrib.layers.batch_norm(h1, center=True, scale=True, is_training=phase, scope='bn')
            return tf.nn.relu(h1, 'relu')
        
    def setup_accuracy(self, output, target):
        with tf.variable_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(target, 1), tf.argmax(output, 1)),
                                      'float32'))
            return accuracy
            
    def setup_loss(self, output, target):
        with tf.variable_scope('loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=target))
            return loss

In [60]:
# Training
tf.reset_default_graph()
with tf.Session() as sess:
    conv_net = ConvNet()
    sess.run(tf.global_variables_initializer())
    train_size = features.shape[0]
    max_epoch = 5
    for epoch in range(max_epoch):
        p = np.random.permutation(features.shape[0])
        for i in range(train_size / 4):
            s = 4*i
            e = s + 4
            curr_batch, curr_labels = get_batch(32, features[p[s:e]], labels[p[s:e]])

            # Train model with batch
            sess.run(conv_net.train_op, feed_dict={
                conv_net.x: curr_batch,
                conv_net.y: curr_labels,
                conv_net.phase: 1
            })
        print(sess.run(conv_net.loss, feed_dict={
            conv_net.x: features,
            conv_net.y: labels,
            conv_net.phase: 1
        }))
        print(sess.run(conv_net.accuracy, feed_dict={
            conv_net.x: features,
            conv_net.y: labels,
            conv_net.phase: 1
        }))
        print(sess.run(conv_net.accuracy, feed_dict={
            conv_net.x: red_features_tr,
            conv_net.y: red_labels_tr,
            conv_net.phase: 1
        }))
        print(sess.run(conv_net.accuracy, feed_dict={
            conv_net.x: red_features_ts,
            conv_net.y: red_labels_ts,
            conv_net.phase: 1
        }))

0.931744
0.566792
0.0
0.5
0.7695
0.683521
0.0
0.9
0.381248
0.856742
0.0
1.0
0.292285
0.883895
0.0
1.0
0.253531
0.915106
1.0
1.0
